#### Evolution Node 
- Task, Input, Output
- LLM guided evolution
- LLM-engineered Code Function or Prompt

Note: 
* Ok to have error, as evolution is decoupled with evaluation for now. 

In [1]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.eoh_evolution import EvolNode


# Code + Compilor Task
mp = MetaPrompt("Generate Fibonacci sequence.", "fibonacci", ["n"], ["sequence"], ["int"], ["list"], PromptMode.CODE) # 
node = EvolNode(mp, None, None)
reasoning, code = node._evolve("i1", replace=True)
input_dict = {"n": 10}

node(input_dict) # Ok we need a output dictionary here as well ...


# Prompt + LLM Task
# mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 
# node = EvolNode(mp, None, None)
# reasoning, code = node._evolve("i1", replace=True)
# input_dict = {"name": "Dilireba"}
# node(input_dict)

Could not load vllm class, check CUDA support and GPU RAM size


ModuleNotFoundError: No module named 'List'

In [2]:
print(node.code)

import List

def fibonacci(n: int) -> List[int]:
    if n <= 0:
        return []
    
    sequence = [0, 1]
    for i in range(2, n):
        sequence.append(sequence[-1] + sequence[-2])
    
    return sequence[:n]


#### Evolution Graph
- Take care of connecting each components (unfolding dictionary object etc.)
- Planning topology with evolution node

In [2]:
from methods.meta_prompt import MetaPlan, extract_json_from_text

mp = MetaPlan("Help me date Dilireba, I'm a white colar worker living on salary.")
prompt = mp._get_prompt_i1()
print("MetaPrompt I1: ")
print(prompt)

from methods.llm import get_openai_response
response = get_openai_response(prompt)

MetaPrompt I1: 
Goal: Help me date Dilireba, I'm a white colar worker living on salary.
First, describe the intuition for your tactics and main steps in one sentence. The description must be inside a brace.
Generate a JSON-style plan represented as a Directed Acyclic Graph (DAG) to achieve the goal. Use creative topology in the DAG, include parallel tasks if required.

The plan should include:

- **Nodes**: Each node represents a key action or step and must contain the following attributes:
  - `task`: Description of the task.
  - `name`: Concise name used for the task function.
  - `input`: The resources, information, or prerequisites needed to perform the action.
  - `output`: The immediate result or outcome of the action.
  - `target`: The purpose or goal that the action contributes to.
  - `mode`: The execution mode for this task ("CODE" or "PROMPT").

- **Edges**: Each edge represents a dependency or relationship between nodes, indicating that one step supports or leads to another

In [11]:
import re 
# from methods.meta_prompt import parse_json_from_response

tactic = re.findall(r"\{(.*)\}", response, re.DOTALL)
plan_dict = extract_json_from_text(response)

In [5]:
from methods.meta_prompt import MetaPrompt, MetaPlan, extract_json_from_text
from methods.eoh_evolution import EvolNode
from methods.llm import get_openai_response as get_response
from typing import Optional, List, Dict

# Collect MetaPrompt from parsed Plan-Graph Response
meta_node_prompts = []
for node in plan_dict["nodes"]:
    node_prompt = MetaPrompt(task=node.get("task"),  func_name=node.get("name"), input=node.get("input"), output=node.get("output"), mode=node.get("mode").lower())
    meta_node_prompts.append(node_prompt)
    
edges = plan_dict["edges"]

EvolNode(meta_prompt = node_prompt)


#### EvalNode Test: 
At the beginning level, generation of EvalNode (when code is used here) should compile with success, and it should takes input and produce output. 